# 3. Pre-processing Training & Data Development

Random Forests are generally not sensitive to feature scales and are generally capable of handling high-dimensional data, Therfore, scaling and dimension reduction was not performed at this time. Also, no data was missing so no values were imputed.

In [1]:
# Libraries imported for this notebook.

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score

In [2]:
# Read EDA_Data.xlsx into a dataframe, formatted dates, and indexed dates.

df = pd.read_excel('/Users/NJahns/Desktop/Bootcamp/Capstone_Two/Edited_Data/EDA_Data.xlsx', parse_dates=True, index_col=[0])

In [3]:
# Looked at shape

df.shape

(2281, 20)

In [4]:
# Printed columns in data

for col in df.columns:
    print(col)

Sludge Volume Index
SE NH3
MLSS
RAS TSS
Alum Feed 1
Alum Feed 2
WAS Flow
Primary Sludge A Flow
Primary Sludge B Flow
Ana F:M
RAS Flow
Avg ND Effluent Hourly Ammonia
Avg ND WAS Flow
Avg ND Aeration DO Cell 5 AVG
Avg ND Aeration DO Cell 7 AVG
Avg ND Aer. Avg Cell 7 Nitrate
Avg ND Average Basin Airflow
day
month
year


## Feature Engineering

### Domain-specific

In [5]:
# Changed 'WAS Flow' to Sludge flow to avoid confusion with 'Avg ND WAS Flow'

df.rename(columns={'WAS Flow': 'Activated Sludge Flow'}, inplace=True)

In [6]:
# Added a column that is 'Total Alum Feed'

df['Total Alum Feed'] = df['Alum Feed 1'] + df['Alum Feed 2']

In [7]:
# Added a column that is 'Total Sludge Flow'

df['Total Primary Sludge Flow'] = df['Primary Sludge A Flow'] + df['Primary Sludge B Flow']

In [8]:
# Added a column that is 'Average ND Aeration DO in Cells'

df['Average ND Aeration DO in Cells'] = df[['Avg ND Aeration DO Cell 5 AVG', 'Avg ND Aeration DO Cell 7 AVG']].mean(axis=1)

In [9]:
# Added a column that is 'MVLSS' - Mixed Liquor Volatile Suspended Solids. The volotile fraction is roughly 75%.

df['MVLSS'] = df['MLSS'] * 0.75

In [10]:
# Added a column that is 'MCRT' - Mean Cell Residence Time

df['MCRT'] = df['MVLSS'] / (df['Activated Sludge Flow'] + df['RAS Flow'])

In [11]:
# Added a column that is 'Sludge Age'

df['Sludge Age'] = df['MCRT'] / (1 + df['Ana F:M'])

### Temporal

Temporal features such as day of the week, month, and year were created in previous steps.

#### Differencing

Differencing was performed to compute the differences between consecutive data points. This technique is ofter perfomed for time series models and can transform a non-stationary time series into a stationary one and remove trends and seasonality. Overall, differencing can be beneficial because it stabilizes the mean and variance of the data, making it easier to identify underlying patterns and relationships.

In [12]:
# Performed differencing on data.

# Identified columns to exclude from differencing
exclude_columns = ['day', 'month', 'year']

# Performed first-order differencing on selected columns
first_differences = df.drop(columns=exclude_columns).diff().add_prefix('1st_')

# Performed second-order differencing on selected columns
second_differences = df.drop(columns=exclude_columns).diff().diff().add_prefix('2nd_')

# Concatenated the results with the original DataFrame
df = pd.concat([df, first_differences, second_differences], axis=1).dropna()

In [13]:
# Looked at just SVI to check differencing calculations.

selected_columns = ['Sludge Volume Index', '1st_Sludge Volume Index', '2nd_Sludge Volume Index']
new_df = df.loc[:, selected_columns]
new_df.head(10)

,Sludge Volume Index,1st_Sludge Volume Index,2nd_Sludge Volume Index
2017-01-03,105.593607,0.000000,0.000000e+00
2017-01-04,105.593607,0.000000,0.000000e+00
2017-01-05,113.661202,8.067595,8.067595e+00
2017-01-06,104.875283,-8.785919,-1.685351e+01
2017-01-07,115.632313,10.757029,1.954295e+01
2017-01-08,126.389342,10.757029,-9.947598e-14
2017-01-09,137.146371,10.757029,8.526513e-14
2017-01-10,114.124294,-23.022078,-3.377911e+01
2017-01-11,104.994903,-9.129391,1.389269e+01
2017-01-12,91.185410,-13.809493,-4.680102e+00


In [14]:
# Checked feature names

for column_title in df.columns:
    print(column_title)

Sludge Volume Index
SE NH3
MLSS
RAS TSS
Alum Feed 1
Alum Feed 2
Activated Sludge Flow
Primary Sludge A Flow
Primary Sludge B Flow
Ana F:M
RAS Flow
Avg ND Effluent Hourly Ammonia
Avg ND WAS Flow
Avg ND Aeration DO Cell 5 AVG
Avg ND Aeration DO Cell 7 AVG
Avg ND Aer. Avg Cell 7 Nitrate
Avg ND Average Basin Airflow
day
month
year
Total Alum Feed
Total Primary Sludge Flow
Average ND Aeration DO in Cells
MVLSS
MCRT
Sludge Age
1st_Sludge Volume Index
1st_SE NH3
1st_MLSS
1st_RAS TSS
1st_Alum Feed 1
1st_Alum Feed 2
1st_Activated Sludge Flow
1st_Primary Sludge A Flow
1st_Primary Sludge B Flow
1st_Ana F:M
1st_RAS Flow
1st_Avg ND Effluent Hourly Ammonia
1st_Avg ND WAS Flow
1st_Avg ND Aeration DO Cell 5 AVG
1st_Avg ND Aeration DO Cell 7 AVG
1st_Avg ND Aer. Avg Cell 7 Nitrate
1st_Avg ND Average Basin Airflow
1st_Total Alum Feed
1st_Total Primary Sludge Flow
1st_Average ND Aeration DO in Cells
1st_MVLSS
1st_MCRT
1st_Sludge Age
2nd_Sludge Volume Index
2nd_SE NH3
2nd_MLSS
2nd_RAS TSS
2nd_Alum Feed 1
2

#### Lag features

I created lagged versions of metrics of one through seven days since SVI tends to change over the course of weeks.

In [15]:
# Lagged features by 1 through 7 days.

explanatory_vars = df.columns[~df.columns.isin(['day', 'month', 'year'])]
lagged_columns = []

lagged_df = pd.DataFrame()

for lag in range(1, 8):
    lagged_columns.extend([f'{var}_lag_{lag}' for var in explanatory_vars])
    lagged_data = {f"{var}_lag_{lag}": df[var].shift(lag) for var in explanatory_vars}
    lagged_df = pd.concat([lagged_df, pd.DataFrame(lagged_data)], axis=1)

df = pd.concat([df, lagged_df], axis=1)

In [16]:
# Dropped rows containng NaN as a result of lagged features.

df.dropna(inplace=True)

In [17]:
# Looked at just SVI to check lagging.

selected_columns = ['Sludge Volume Index', 'Sludge Volume Index_lag_1', 'Sludge Volume Index_lag_2', 'Sludge Volume Index_lag_3', 'Sludge Volume Index_lag_4']
new_df = df.loc[:, selected_columns]
new_df.head(10)

,Sludge Volume Index,Sludge Volume Index_lag_1,Sludge Volume Index_lag_2,Sludge Volume Index_lag_3,Sludge Volume Index_lag_4
2017-01-10,114.124294,137.146371,126.389342,115.632313,104.875283
2017-01-11,104.994903,114.124294,137.146371,126.389342,115.632313
2017-01-12,91.185410,104.994903,114.124294,137.146371,126.389342
2017-01-13,93.179635,91.185410,104.994903,114.124294,137.146371
2017-01-14,94.544590,93.179635,91.185410,104.994903,114.124294
2017-01-15,95.909545,94.544590,93.179635,91.185410,104.994903
2017-01-16,97.274501,95.909545,94.544590,93.179635,91.185410
2017-01-17,98.639456,97.274501,95.909545,94.544590,93.179635
2017-01-18,118.541033,98.639456,97.274501,95.909545,94.544590
2017-01-19,106.382979,118.541033,98.639456,97.274501,95.909545


In [18]:
# Printed list of column titles to check results of lagging.

for column_title in df.columns:
    print(column_title)

Sludge Volume Index
SE NH3
MLSS
RAS TSS
Alum Feed 1
Alum Feed 2
Activated Sludge Flow
Primary Sludge A Flow
Primary Sludge B Flow
Ana F:M
RAS Flow
Avg ND Effluent Hourly Ammonia
Avg ND WAS Flow
Avg ND Aeration DO Cell 5 AVG
Avg ND Aeration DO Cell 7 AVG
Avg ND Aer. Avg Cell 7 Nitrate
Avg ND Average Basin Airflow
day
month
year
Total Alum Feed
Total Primary Sludge Flow
Average ND Aeration DO in Cells
MVLSS
MCRT
Sludge Age
1st_Sludge Volume Index
1st_SE NH3
1st_MLSS
1st_RAS TSS
1st_Alum Feed 1
1st_Alum Feed 2
1st_Activated Sludge Flow
1st_Primary Sludge A Flow
1st_Primary Sludge B Flow
1st_Ana F:M
1st_RAS Flow
1st_Avg ND Effluent Hourly Ammonia
1st_Avg ND WAS Flow
1st_Avg ND Aeration DO Cell 5 AVG
1st_Avg ND Aeration DO Cell 7 AVG
1st_Avg ND Aer. Avg Cell 7 Nitrate
1st_Avg ND Average Basin Airflow
1st_Total Alum Feed
1st_Total Primary Sludge Flow
1st_Average ND Aeration DO in Cells
1st_MVLSS
1st_MCRT
1st_Sludge Age
2nd_Sludge Volume Index
2nd_SE NH3
2nd_MLSS
2nd_RAS TSS
2nd_Alum Feed 1
2

In [19]:
# Saved to Excel
df.to_excel('/Users/NJahns/Desktop/Bootcamp/Capstone_Two/Edited_Data/Pre_Process_Train.xlsx', index=True)

I did not include splitting the data into testing and training datasets in this workbook becasue it makes more sense to include those steps with model development as they are mixed into the model development steps.